In [1]:
import sys, os
import re
import cv2
import matplotlib.pyplot as plt
sys.path.append("../")
from lib.check import check_paths
paths = check_paths()

Check completed!


In [2]:
# # .jpeg to .pdf converter
# from PIL import Image
# import os

# image = Image.open(paths['data_path']/'Image.jpeg')
# pdf_path = paths['out_path']/'Image.pdf'
# image.save(pdf_path, 'PDF', resolution=100.0)

In [3]:
from PyPDF2 import PdfReader
pdf = PdfReader(paths['data_path']/'comprobante.pdf')
text = ""
for page in pdf.pages:
    text += page.extract_text()
if "CFE" in text:
    try:
        parts = text.split("Servicios Básicos")
        if len(parts) > 1:
            address_block = parts[1].strip()
            lines = [line.strip() for line in address_block.split('\n') if line.strip()]
            
            if len(lines) >= 2:
                line0 = lines[0]
                line1 = "".join(lines[1].split(",")[1:2]).strip()
                import re
                line1 = re.sub(r'[^\d]', '', line1)
                line2 = lines[2] if len(lines) > 2 else ""

                if ", colonia" in line0:
                    street, colony = line0.split(", colonia")
                    colony = "colonia" + colony
                else:
                    street = line0
                    colony = ""
                
                cp_line = line1
                
                city = line2.split(".")[0]
                
                domicile = [street.strip(), colony.strip(', '), cp_line.strip(', '), city.strip()]
                print("Found Domicile from PDF:")
                print(f"Street: {domicile[0]}")
                print(f"Colony: {domicile[1]}")
                print(f"CP Line: {domicile[2]}")
                print(f"City: {domicile[3]}")
            else:
                 print("Address lines not found in expected format.")
    except Exception as e:
        print(f"Error parsing address: {e}")


Found Domicile from PDF:
Street: Río Ródano No. 14
Colony: colonia Cuauhtémoc
CP Line: 06500
City: Ciudad de México


In [4]:
from PIL import Image
import pytesseract

try:
    image_path = paths['data_path'] / 'Image.jpeg'
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    #print(text)

    lines = [line.strip() for line in text.split('\n') if line.strip()]
    domicile = []

    for i, line in enumerate(lines):
        if "C.P." in line:
            if i >= 2:
                street = lines[i-2].split('|')[0].strip()
                colony = lines[i-1]
                cp_city = " ".join(line.split(' ')[1:2]).strip().rstrip(",")
                
                city = ""
                if i + 1 < len(lines) and "Estado de cuenta" not in lines[i+1]:
                     city = lines[i+1]
                elif i + 2 < len(lines) and "Estado de cuenta" not in lines[i+2]:
                     city = " ".join(lines[i+2].split(' ')[1:3])
                
                domicile = [street, colony, cp_city, city]
                print("Found Domicile from Image:")
                print(f"Street: {street}")
                print(f"Colony: {colony}")
                print(f"CP Line: {cp_city}")
                print(f"City: {city}")
                break
    
    if not domicile:
        print("Could not find domicile (C.P. anchor).")

except Exception as e:
    print(f"Error: {e}")


Found Domicile from Image:
Street: CIRO B CEBALLOS 67
Colony: COLONIAL IZTAPALAPA
CP Line: 09270
City: Distrito Federal


In [11]:
# from structocr import StructOCR
# from dotenv import load_dotenv
# load_dotenv(override=True)

# client = StructOCR(os.getenv("STRUCT_OCR_API_KEY"))
# def scan_mexico_id():
#     image_path = paths["data_path"]/"ine zai.pdf"
#     # pdf to image
#     from pdf2image import convert_from_path
#     pages = convert_from_path(image_path)
#     for i, page in enumerate(pages):
#         page.save(paths["out_path"]/f"ine_{i}.jpeg", 'JPEG')
#     try:
#         print(f"Scanning {image_path}...")
        
#         result = client.scan_national_id(paths["out_path"]/f"ine_0.jpeg")

#         if result.get('success'):
#             data = result['data']
#             print("México Extraction Successful!")
            
#             print(f"Region:      {data.get('country_code')} (Series: {data.get('card_series')})")
#             print(f"Name:        {data.get('given_names')} {data.get('surname')}")
#             print(f"ID Number:   {data.get('document_number')}")
            
#             print(f"Personal #:  {data.get('personal_number')}")
            
#             print(f"DOB:         {data.get('date_of_birth')} ({data.get('sex')})")
#             print(f"Address:     {data.get('address')}")
#         else:
#             print(f"Extraction Failed: {result.get('error')}")

#     except Exception as e:
#         print(f"An error occurred: {e}")

# if __name__ == "__main__":
#     scan_mexico_id()

In [ ]:
# R7126125